<a href="https://colab.research.google.com/github/AnastasiaSperanskaya/MachineLearningITMO/blob/main/midtermTests/midtermTest2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Рубежная контрольная работа

Эту работу выполнять не обязательно. Баллы набранные за работу просуммируются с баллами, которые были набраны за контрольную работу 1 и выставлены как оценка за промежуточную аттестацию "автоматом". 

Сделайте копию ноутбука

Присвойте переменной `v` значение свой табельный номер ИСУ, это нужно для определения вашего варианта. Запустите код в следующей ячейке.

In [ ]:
from datetime import datetime
current_datetime = datetime.now()
print(current_datetime)
v = 264248 # по умолчанию вписан мой номер

2020-12-21 15:14:16.097253


# Общее задание

Нужно решить задачу бинарной классификации, предварительно построив признаковое описание объектов на основе нескольких таблиц. 

Целевая переменная - пол клиента.

В качестве модели нужно использовать нейронную сеть, которую нужно строить с помощью `keras` или `torch` на выбор студента.

# Данные

Для всех последующих заданий будем использовать обезличенные транзакционные банковские данные. Для этого считайте в переменные **transactions, tr_mcc_codes и gender_train** из одноимённых таблиц из папки data. Для таблицы transactions используйте только первые n=1000000 строк.

Эти данные уже использовались в работе 2. Описание данных тут: https://drive.google.com/file/d/13sn8orjkoGfO1SVZYByXNmZQYFRfmOdp/view?usp=sharing

Скачать данные отсюда: https://drive.google.com/drive/folders/1YAMe7MiTxA-RSSd8Ex2p-L0Dspe6Gs4L?usp=sharing

## Задание 1

В задании требуется на основе нескольких таблиц с данными сделать признаковое описание объектов. 

Объектами являются клиенты. Клиенты идентифицируются с помощью `customer_id`, которые есть в таблицах **transactions** и **gender_train**. В качестве призаков нужно использовать даннее по категориям транзаций: ```mcc_code``` — mcc-код транзакции есть в таблицах **transactions** и **tr_mcc_codes**


Для выбора варианта, запустите код в следующей ячейке



In [ ]:
print ('Список категорий')
feature_list = ['Одежда (также нужно искать "одежды")', 'Аптеки', 'Ценные бумаги', 'Магазины косметики и Салоны красоты', 'Образовательные услуги (а также школы, университеты)',
                'Алименты', 'Прокат автомобилей', 'Автозапчасти, Автошины и Станции техобслуживания (а также СТО)', 'Видеоигры (искать надо по "видеоигр")',
                'Отели', 'Программирование', 'Ремонт', 'Предсказатели будущего', 'Ювелирные изделия']
if (int(str(v)[4]) % 2) == 0:
  print (feature_list[::2])
else:
  print (feature_list[::-2])


if (int(str(v)[5]) % 2) == 0:
  print ('В качестве значений признаков используйте количество транзакций с тратами и количество транзакций с поступлениями')
else: 
  print ('В качестве значений признаков используйте сумму по транзакциям с тратами и сумму по транзакциям с поступлениями')

Список категорий
['Одежда (также нужно искать "одежды")', 'Ценные бумаги', 'Образовательные услуги (а также школы, университеты)', 'Прокат автомобилей', 'Видеоигры (искать надо по "видеоигр")', 'Программирование', 'Предсказатели будущего']
В качестве значений признаков используйте количество транзакций с тратами и количество транзакций с поступлениями


Каждая категория может быть связана с одним или несколькими mcc-кодами. Например, вот несколько mcc-кодов по категории "Одежда": 
* 5621; Готовая женская одежда
* 5641; Детская одежда
* 5651; Одежда для всей семьи

Таким образом получится 14 признаков, то есть семь категорий, каждая из которых даст два значения: траты и поступления. Например, категория "Одежда" даёт признаки "Одежда траты" и "Одежда поступления".

Сделайте нормализацию значений признаков так, чтобы все означения менялись от 0 до 1. 

In [ ]:
!pip install PyDrive
import pandas as pd
import numpy as np
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
downloaded = drive.CreateFile({'id':"1glFA40HUSGvjxCtyqSmPYpXbPNY9WOJM"})
downloaded.GetContentFile('transactions.csv')
transactions = pd.read_csv('transactions.csv', sep = ',', nrows = 1000000)
downloaded = drive.CreateFile({'id':"1t6ij-5D8v_VXFn8RKAOL5sn3qDWCMiiF"})
downloaded.GetContentFile('tr_mcc_codes.csv')
tr_mcc_codes = pd.read_csv('tr_mcc_codes.csv', sep = ';')
downloaded = drive.CreateFile({'id':"1fd81mF4KWKm-e7xCqDAVtHN5T5yn2k_D"})
downloaded.GetContentFile('gender_train.csv')
gender_train = pd.read_csv('gender_train.csv', sep = ',')

In [ ]:
mcc_clothes = tr_mcc_codes[tr_mcc_codes['mcc_description'].str.contains('Одежда|одежды')]
mcc_clothes

,mcc_code,mcc_description
73,5651,Одежда для всей семьи
77,5691,Магазины мужской и женской одежды
78,5697,"Услуги по переделке, починке и пошиву одежды"
79,5699,Различные магазины одежды и аксессуаров


In [ ]:
mcc_papers = tr_mcc_codes[tr_mcc_codes['mcc_description'].str.contains('Ценные бумаги')]
mcc_papers

,mcc_code,mcc_description
127,6211,Ценные бумаги: брокеры/дилеры


In [ ]:
mcc_edu = tr_mcc_codes[tr_mcc_codes['mcc_description'].str.contains('Образовательные услуги|школы|университеты')]
mcc_edu

,mcc_code,mcc_description
172,8220,"Колледжи, университеты, профессиональные школы..."
174,8299,Образовательные услуги


In [ ]:
mcc_auto = tr_mcc_codes[tr_mcc_codes['mcc_description'].str.contains('Прокат автомобилей')]
mcc_auto

,mcc_code,mcc_description
146,7512,Прокат автомобилей


In [ ]:
mcc_games = tr_mcc_codes[tr_mcc_codes['mcc_description'].str.contains('видеоигр')]
mcc_games

,mcc_code,mcc_description
160,7993,Принадлежности для видеоигр
161,7994,Галереи/учреждения видеоигр


In [ ]:
mcc_code = tr_mcc_codes[tr_mcc_codes['mcc_description'].str.contains('Программирование')]
mcc_code

,mcc_code,mcc_description
142,7372,"Программирование, обработка данных, интегриров..."


In [ ]:
mcc_future = tr_mcc_codes[tr_mcc_codes['mcc_description'].str.contains('предсказатели будущего')]
mcc_future

,mcc_code,mcc_description
163,7996,"Луна-парки, карнавалы, цирки, предсказатели бу..."


In [ ]:
df = pd.DataFrame(columns=['id', 'clothes_spend', 'clothes_addm', 'papers_spend', 'papers_addm', 'edu_spend', 'edu_addm', 'auto_spend', 'auto_addm', 'games_spend', 'games_addm', 'code_spend', 'code_addm', 'future_spend', 'future_addm', 'gender'])
df['gender'] = gender_train['gender']
df['id'] = gender_train['customer_id']
table = pd.DataFrame(columns=['id', 'clothes_spend', 'clothes_addm', 'papers_spend', 'papers_addm', 'edu_spend', 'edu_addm', 'auto_spend', 'auto_addm', 'games_spend', 'games_addm', 'code_spend', 'code_addm', 'future_spend', 'future_addm', 'gender'])

In [ ]:
import warnings
warnings.filterwarnings("ignore")
customers = np.array(gender_train['customer_id'])
for customer in customers:
  row = df.loc[df['id'] == customer]

  smpl = transactions.loc[transactions['customer_id'] == customer]

  smpl_attr = smpl.loc[smpl['mcc_code'].isin(mcc_clothes['mcc_code'])]
  smpl_attr_addm = smpl_attr.loc[smpl_attr['amount'] > 0]
  smpl_attr_spend = smpl_attr.loc[smpl_attr['amount'] < 0]
  row['clothes_spend'] = smpl_attr_spend.shape[0]
  row['clothes_addm'] = smpl_attr_addm.shape[0]

  smpl_attr = smpl.loc[smpl['mcc_code'].isin(mcc_papers['mcc_code'])]
  smpl_attr_addm = smpl_attr.loc[smpl_attr['amount'] > 0]
  smpl_attr_spend = smpl_attr.loc[smpl_attr['amount'] < 0]
  row['papers_spend'] = smpl_attr_spend.shape[0]
  row['papers_addm'] = smpl_attr_addm.shape[0]

  smpl_attr = smpl.loc[smpl['mcc_code'].isin(mcc_edu['mcc_code'])]
  smpl_attr_addm = smpl_attr.loc[smpl_attr['amount'] > 0]
  smpl_attr_spend = smpl_attr.loc[smpl_attr['amount'] < 0]
  row['edu_spend'] = smpl_attr_spend.shape[0]
  row['edu_addm'] = smpl_attr_addm.shape[0]

  smpl_attr = smpl.loc[smpl['mcc_code'].isin(mcc_auto['mcc_code'])]
  smpl_attr_addm = smpl_attr.loc[smpl_attr['amount'] > 0]
  smpl_attr_spend = smpl_attr.loc[smpl_attr['amount'] < 0]
  row['auto_spend'] = smpl_attr_spend.shape[0]
  row['auto_addm'] = smpl_attr_addm.shape[0]

  smpl_attr = smpl.loc[smpl['mcc_code'].isin(mcc_games['mcc_code'])]
  smpl_attr_addm = smpl_attr.loc[smpl_attr['amount'] > 0]
  smpl_attr_spend = smpl_attr.loc[smpl_attr['amount'] < 0]
  row['games_spend'] = smpl_attr_spend.shape[0]
  row['games_addm'] = smpl_attr_addm.shape[0]

  smpl_attr = smpl.loc[smpl['mcc_code'].isin(mcc_code['mcc_code'])]
  smpl_attr_addm = smpl_attr.loc[smpl_attr['amount'] > 0]
  smpl_attr_spend = smpl_attr.loc[smpl_attr['amount'] < 0]
  row['code_spend'] = smpl_attr_spend.shape[0]
  row['code_addm'] = smpl_attr_addm.shape[0]

  smpl_attr = smpl.loc[smpl['mcc_code'].isin(mcc_future['mcc_code'])]
  smpl_attr_addm = smpl_attr.loc[smpl_attr['amount'] > 0]
  smpl_attr_spend = smpl_attr.loc[smpl_attr['amount'] < 0]
  row['future_spend'] = smpl_attr_spend.shape[0]
  row['future_addm'] = smpl_attr_addm.shape[0]

  row = row.reset_index(drop=True)
  if (row.iloc[0]['clothes_spend'] != 0) or (row.iloc[0]['clothes_addm'] != 0) or (row.iloc[0]['papers_spend'] != 0) or (row.iloc[0]['papers_addm'] != 0) or (row.iloc[0]['edu_spend'] != 0) or (row.iloc[0]['edu_addm'] != 0) or (row.iloc[0]['auto_spend'] != 0) or (row.iloc[0]['auto_addm'] != 0) or (row.iloc[0]['games_spend'] != 0) or (row.iloc[0]['games_addm'] != 0) or (row.iloc[0]['code_spend'] != 0) or (row.iloc[0]['code_addm'] != 0) or (row.iloc[0]['future_spend'] != 0) or (row.iloc[0]['future_addm'] != 0):
    df.loc[df['id'] == customer] = row
    table = table.append(row)

  #df.loc[df['id'] == customer] = row 


In [ ]:
del table['id']
df1 = table.drop(labels='gender', axis=1)
table = table.reset_index(drop=True)
from sklearn import preprocessing
import pandas as pd
d = preprocessing.normalize(df1, axis = 0)
scaled_df = pd.DataFrame(d, columns=df1.columns)
scaled_df['gender'] = table['gender']
scaled_df.head()

,clothes_spend,clothes_addm,papers_spend,papers_addm,edu_spend,edu_addm,auto_spend,auto_addm,games_spend,games_addm,code_spend,code_addm,future_spend,future_addm,gender
0,0.013214,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,1
1,0.026427,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.26968,0.0,0
2,0.015856,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0
3,0.031713,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0
4,0.010571,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,1


# Задание 2

Модель - многослойная нейронная сеть с двумя скрытыми слоями и с dropout. Количество нейронов в каждом слое выбираете самостоятельно.
Функции активации, метод оптимизации, скорость обучения, вероятность dropout - выбираете самостоятельно

Модель нужно строить с помощью keras или torch

Нужно отдельно перечислить все гиперпараметры и их значения вот в таком формате (значения приведены для примера, у вас могут быть другие):

* количество эпох: 5
* скорость обучения: 0.01
* функция активация: ReLU
* функция потерь: CrossEntropyLoss
* оптимизатор: SGD


Настроить параметры модели.

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(scaled_df.loc[:,'clothes_spend':'future_addm'], scaled_df['gender'], test_size=0.7)
X_train.head()

,clothes_spend,clothes_addm,papers_spend,papers_addm,edu_spend,edu_addm,auto_spend,auto_addm,games_spend,games_addm,code_spend,code_addm,future_spend,future_addm
512,0.013214,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
495,0.002643,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.013214,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
421,0.007928,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
611,0.002643,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
y_test
y_test = y_test.astype(np.float)
y_train = y_train.astype(np.float)

In [ ]:
import torch
import torchvision
Input = 14
Output = 2
mlp = torch.nn.Sequential(
    torch.nn.Linear(Input, 15),
    torch.nn.ReLU(),
    torch.nn.Linear(15, 48),
    torch.nn.ReLU(),
    torch.nn.Dropout(0.25),
    torch.nn.Linear(48, 12),
    torch.nn.ReLU(),
    torch.nn.Dropout(0.75),
    torch.nn.Linear(12, Output),
    torch.nn.ReLU(),
    torch.nn.LogSoftmax(dim=1)
).cuda()

In [ ]:
learning_rate = 0.01
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(mlp.parameters(), lr = learning_rate)
n_epochs = 5

In [ ]:
for epochs in range(n_epochs):
  loss_epoch = 0
  accuracy = 0
  data = torch.FloatTensor(X_train.values)
  target = torch.LongTensor(y_train.values)
  data, target = torch.autograd.Variable(data).cuda(), torch.autograd.Variable(target).cuda()
  data = data.view(-1, Input)
  y_pred = mlp(data)
  loss=loss_fn(y_pred, target)
  optimizer.zero_grad()
  loss.backward()
  optimizer.step()
  loss_epoch += loss
  accuracy_epoch = 0
  data = torch.FloatTensor(X_test.values)
  target = torch.LongTensor(y_test.values)
  data, target = torch.autograd.Variable(data).cuda(), torch.autograd.Variable(target).cuda()
  data = data.view(-1, Input)
  y_pred = mlp(data).cuda()
  predicted = y_pred.max(1)[1]
  c = (predicted == target)
  accuracy_epoch += c.float().sum() / target.shape[0]
  print('Epoch %d, Loss: %.6f, Accuracy: %.3f %%' % (epochs+1, loss_epoch, 100*accuracy_epoch))


Epoch 1, Loss: 0.692149, Accuracy: 59.825 %
Epoch 2, Loss: 0.691618, Accuracy: 59.825 %
Epoch 3, Loss: 0.688881, Accuracy: 59.825 %
Epoch 4, Loss: 0.689845, Accuracy: 59.825 %
Epoch 5, Loss: 0.691791, Accuracy: 59.825 %


# Задание 3


Проверить качество модели не менее чем на трёх разных метриках. Вывести эти значения.

После завершения контрольной работы, дайте ссылку на colab в отдельном followup в своей теме на Piazza. Дайте разрешение на его просмотр.

Не забудьте запустить код в последней ячейке для контроля времени выполнения.

In [ ]:
current_datetime = datetime.now()
print(current_datetime)

2020-12-21 15:14:26.214918
